In [1]:
import numpy as np
import pandas as pd
import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import mlflow
import optuna
import optuna.integration.lightgbm as lgb
import math

In [2]:
def objective(trial):
    train = pd.read_csv("data_transformed.csv") #reading train data
    test = pd.read_csv("data_transformed.csv") #reading test data
    train_x= train.iloc[:1000,:-1]
    test_x= test.iloc[:1000,:-1]
    train_y = train.iloc[:1000,:]['Class'] 
    test_y = test.iloc[:1000,:]['Class']
    sc = StandardScaler() #scaling data
    scaled_X_train = sc.fit_transform(train_x)
    scaled_X_test = sc.transform(test_x)
    dtrain = lgb.Dataset(scaled_X_train, label = train_y, free_raw_data = False)
    dtest  = lgb.Dataset(scaled_X_test, label = test_y, free_raw_data = False)
    
    param = {
        'objective': 'poisson',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'force_row_wise': True,
        'max_depth': -1,
        
        'max_bin': trial.suggest_int('max_bin', 1, 512),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),

        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        
        'sub_feature': trial.suggest_uniform('sub_feature', 0.0, 1.0),
        'sub_row': trial.suggest_uniform('sub_row', 0.0, 1.0)
    }
    
    # Add a callback for pruning
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'rmse')
    
    gbm = lgb.train(
        param, 
        dtrain, 
        verbose_eval = 20,
        valid_sets = [dtest], 
        callbacks = [pruning_callback]
        )
    
    preds = gbm.predict(scaled_X_test)
    accuracy = (sklearn.metrics.mean_squared_error(test_y, preds))

    return accuracy



    

In [ ]:
    study = optuna.create_study(direction = 'minimize', pruner = optuna.pruners.MedianPruner(n_warmup_steps = 10))
    study.optimize(objective, n_trials = 500)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2021-03-15 23:22:24,020] A new study created in memory with name: no-name-048dd9c1-1350-4ea9-9fc0-e55bc72b7d7e
[I 2021-03-15 23:22:36,575] A new study created in memory with name: no-name-0c175484-e6f2-4f69-be49-ce061c2ea715
feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=84, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=84
[20]	valid_0's rmse: 0.0502983
[40]	valid_0's rmse: 0.0266233
[60]	valid_0's rmse: 0.0278232
[80]	valid_0's rmse: 0.0295985
[100]	valid_0's rmse: 0.0151152
[120]	valid_0's rmse: 0.0103312
[140]	valid_0's rmse: 0.0103312
[160]	valid_0's rmse: 0.0103312
[180]	valid_0's rmse: 0.0103312
[200]	valid_0's rmse: 0.0103312
[220]	valid_0's rmse: 0.0103312
[240]	valid_0's rmse: 0.0103312
[260]	valid_0's rmse: 0.0103312
[280]	valid_0's rmse: 0.0103312
[300]	valid_0's rmse: 0.0103312
[320]	valid_0's rmse: 0.0103312
[340]	valid_0's rmse: 0.0103312
[360]	valid_0's rmse: 0.0103312
[380]	valid_0's rmse: 0.0103312
[400]	valid_0's rmse: 0.0103312
[420]	valid_0's rmse: 0.0103312
[440]	valid_0's rmse: 0.0103312
[460]	valid_0's rmse: 0.0103312
[480]	valid_0's rmse: 0.0103312
[500]	valid_0's rmse: 0.0103312
[520]	valid_0's rmse: 0.0103312
[540]	valid_0's rmse: 0.0103312
[560]	valid_0's rm

feature_fraction, val_score: 0.010331:  14%|######4                                      | 1/7 [00:03<00:19,  3.21s/it]

[20]	valid_0's rmse: 0.0479547
[40]	valid_0's rmse: 0.0261161
[60]	valid_0's rmse: 0.0287627
[80]	valid_0's rmse: 0.0279036
[100]	valid_0's rmse: 0.0279036
[120]	valid_0's rmse: 0.0279036
[140]	valid_0's rmse: 0.0279036
[160]	valid_0's rmse: 0.0279036
[180]	valid_0's rmse: 0.0279036
[200]	valid_0's rmse: 0.0279036
[220]	valid_0's rmse: 0.0279036
[240]	valid_0's rmse: 0.0279036
[260]	valid_0's rmse: 0.0279036
[280]	valid_0's rmse: 0.0279036
[300]	valid_0's rmse: 0.0279036
[320]	valid_0's rmse: 0.0279036
[340]	valid_0's rmse: 0.0279036
[360]	valid_0's rmse: 0.0279036
[380]	valid_0's rmse: 0.0279036
[400]	valid_0's rmse: 0.0279036
[420]	valid_0's rmse: 0.0279036
[440]	valid_0's rmse: 0.0279036
[460]	valid_0's rmse: 0.0279036
[480]	valid_0's rmse: 0.0279036
[500]	valid_0's rmse: 0.0279036
[520]	valid_0's rmse: 0.0279036
[540]	valid_0's rmse: 0.0279036
[560]	valid_0's rmse: 0.0279036
[580]	valid_0's rmse: 0.0279036
[600]	valid_0's rmse: 0.0279036
[620]	valid_0's rmse: 0.0279036
[640]	valid_

feature_fraction, val_score: 0.010331:  29%|############8                                | 2/7 [00:05<00:15,  3.02s/it]

...

[20]	valid_0's rmse: 0.0407222
[40]	valid_0's rmse: 0.0265745
[60]	valid_0's rmse: 0.0231915
[80]	valid_0's rmse: 0.0231029
[100]	valid_0's rmse: 0.0134946
[120]	valid_0's rmse: 0.00965064
[140]	valid_0's rmse: 0.0096441
[160]	valid_0's rmse: 0.00984743
[180]	valid_0's rmse: 0.00836473
[200]	valid_0's rmse: 0.00735662
[220]	valid_0's rmse: 0.00663846
[240]	valid_0's rmse: 0.00966708
[260]	valid_0's rmse: 0.00860479
[280]	valid_0's rmse: 0.00800052
[300]	valid_0's rmse: 0.00764112
[320]	valid_0's rmse: 0.00751028
[340]	valid_0's rmse: 0.00690816
[360]	valid_0's rmse: 0.0062652
[380]	valid_0's rmse: 0.00803464
[400]	valid_0's rmse: 0.00856014
[420]	valid_0's rmse: 0.00785033
[440]	valid_0's rmse: 0.00753368
[460]	valid_0's rmse: 0.00668233
[480]	valid_0's rmse: 0.00725967
[500]	valid_0's rmse: 0.00763537
[520]	valid_0's rmse: 0.00746569
[540]	valid_0's rmse: 0.0069954
[560]	valid_0's rmse: 0.00717627
[580]	valid_0's rmse: 0.0062761
[600]	valid_0's rmse: 0.00576274
[620]	valid_0's rmse: 0

feature_fraction_stage2, val_score: 0.005132:  17%|######3                               | 1/6 [00:03<00:18,  3.78s/it]

[20]	valid_0's rmse: 0.0407638
[40]	valid_0's rmse: 0.0257093
[60]	valid_0's rmse: 0.0227738
[80]	valid_0's rmse: 0.0231445
[100]	valid_0's rmse: 0.01377
[120]	valid_0's rmse: 0.0101201
[140]	valid_0's rmse: 0.00949794
[160]	valid_0's rmse: 0.0100406
[180]	valid_0's rmse: 0.0087769
[200]	valid_0's rmse: 0.00799882
[220]	valid_0's rmse: 0.00851101
[240]	valid_0's rmse: 0.00851101
[260]	valid_0's rmse: 0.00851101
[280]	valid_0's rmse: 0.00851101
[300]	valid_0's rmse: 0.00851101
[320]	valid_0's rmse: 0.00851101
[340]	valid_0's rmse: 0.00851101
[360]	valid_0's rmse: 0.00851101
[380]	valid_0's rmse: 0.00851101
[400]	valid_0's rmse: 0.00851101
[420]	valid_0's rmse: 0.00851101
[440]	valid_0's rmse: 0.00851101
[460]	valid_0's rmse: 0.00851101
[480]	valid_0's rmse: 0.00851101
[500]	valid_0's rmse: 0.00851101
[520]	valid_0's rmse: 0.00851101
[540]	valid_0's rmse: 0.00851101
[560]	valid_0's rmse: 0.00851101
[580]	valid_0's rmse: 0.00851101
[600]	valid_0's rmse: 0.00851101
[620]	valid_0's rmse: 0.

feature_fraction_stage2, val_score: 0.005132:  33%|############6                         | 2/6 [00:05<00:12,  3.10s/it]

[20]	valid_0's rmse: 0.0408141
[40]	valid_0's rmse: 0.0266075
[60]	valid_0's rmse: 0.0232244
[80]	valid_0's rmse: 0.0235395
[100]	valid_0's rmse: 0.0136813
[120]	valid_0's rmse: 0.00987356
[140]	valid_0's rmse: 0.00972612
[160]	valid_0's rmse: 0.0103046
[180]	valid_0's rmse: 0.00851788
[200]	valid_0's rmse: 0.00748197
[220]	valid_0's rmse: 0.00675971
[240]	valid_0's rmse: 0.00953941
[260]	valid_0's rmse: 0.00869565
[280]	valid_0's rmse: 0.00803802
[300]	valid_0's rmse: 0.00766919
[320]	valid_0's rmse: 0.00754836
[340]	valid_0's rmse: 0.00695975
[360]	valid_0's rmse: 0.00630566
[380]	valid_0's rmse: 0.00810549
[400]	valid_0's rmse: 0.00861241
[420]	valid_0's rmse: 0.0079295
[440]	valid_0's rmse: 0.00761187
[460]	valid_0's rmse: 0.00715204
[480]	valid_0's rmse: 0.00727118
[500]	valid_0's rmse: 0.00740358
[520]	valid_0's rmse: 0.00716582
[540]	valid_0's rmse: 0.00723993
